In [34]:
import os
import numpy as np
import csv
from pathlib import Path
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
csvpath = Path("comments_5_8_2.csv")

In [3]:
data = pd.read_csv(csvpath, error_bad_lines=False,)
data

,Unnamed: 0,Author,Comment,Score,timestamp
0,0,None,"all right, bears. its been fun, but im out",87,2020-05-08 07:31:36
1,1,moonski,Bloomberg - Markets Brace for Catastrophic Job...,71,2020-05-08 06:14:25
2,2,None,Literally the worst jobs report in history goi...,60,2020-05-08 05:26:53
3,3,visionarymind,magine returning to work to find out:\n\n1) sa...,106,2020-05-08 05:42:06
4,4,niceguy897,I WAS PROMISED A TOTAL FINANCIAL COLLAPSE WHER...,51,2020-05-08 05:26:46
...,...,...,...,...,...
1704,424,None,"Luckin Coffee fires CEO, COO after sales fraud...",7,2020-05-12 10:05:16
1705,425,brownbrandon,post that GRUB gain porn plz,6,2020-05-12 10:08:23
1706,426,kckckc111,Uber discussing buying grub thats why it shot up,7,2020-05-12 10:09:21
1707,427,LMY723,Uber puts. Grubhub acquisition gonna be pricey.,6,2020-05-12 10:15:27


In [4]:
# word tokenize all sentences
word_tokenized = []

for Comment in data:
    # get all for each article, which is already sentence tokenized
    words = []
    for sent in Comment:
        words = words + word_tokenize(sent)
    # append all words for each article to the word_tokenized list
    word_tokenized.append(words)

In [5]:
reuters_cpi = pd.DataFrame({'word_tokenized': word_tokenized})

In [6]:
reuters_cpi

,word_tokenized
0,"[U, n, n, a, m, e, d, :, 0]"
1,"[A, u, t, h, o, r]"
2,"[C, o, m, m, e, n, t]"
3,"[S, c, o, r, e]"
4,"[t, i, m, e, s, t, a, m, p]"


In [7]:
data['tokenized_sents'] = data.apply(lambda column: nltk.word_tokenize(column['Comment']), axis=1)
data

,Unnamed: 0,Author,Comment,Score,timestamp,tokenized_sents
0,0,None,"all right, bears. its been fun, but im out",87,2020-05-08 07:31:36,"[all, right, ,, bears, ., its, been, fun, ,, b..."
1,1,moonski,Bloomberg - Markets Brace for Catastrophic Job...,71,2020-05-08 06:14:25,"[Bloomberg, -, Markets, Brace, for, Catastroph..."
2,2,None,Literally the worst jobs report in history goi...,60,2020-05-08 05:26:53,"[Literally, the, worst, jobs, report, in, hist..."
3,3,visionarymind,magine returning to work to find out:\n\n1) sa...,106,2020-05-08 05:42:06,"[magine, returning, to, work, to, find, out, :..."
4,4,niceguy897,I WAS PROMISED A TOTAL FINANCIAL COLLAPSE WHER...,51,2020-05-08 05:26:46,"[I, WAS, PROMISED, A, TOTAL, FINANCIAL, COLLAP..."
...,...,...,...,...,...,...
1704,424,None,"Luckin Coffee fires CEO, COO after sales fraud...",7,2020-05-12 10:05:16,"[Luckin, Coffee, fires, CEO, ,, COO, after, sa..."
1705,425,brownbrandon,post that GRUB gain porn plz,6,2020-05-12 10:08:23,"[post, that, GRUB, gain, porn, plz]"
1706,426,kckckc111,Uber discussing buying grub thats why it shot up,7,2020-05-12 10:09:21,"[Uber, discussing, buying, grub, thats, why, i..."
1707,427,LMY723,Uber puts. Grubhub acquisition gonna be pricey.,6,2020-05-12 10:15:27,"[Uber, puts, ., Grubhub, acquisition, gon, na,..."


In [13]:
comments_stuff = data["tokenized_sents"]

In [14]:
print(comments_stuff)

0       [all, right, ,, bears, ., its, been, fun, ,, b...
1       [Bloomberg, -, Markets, Brace, for, Catastroph...
2       [Literally, the, worst, jobs, report, in, hist...
3       [magine, returning, to, work, to, find, out, :...
4       [I, WAS, PROMISED, A, TOTAL, FINANCIAL, COLLAP...
                              ...                        
1704    [Luckin, Coffee, fires, CEO, ,, COO, after, sa...
1705                  [post, that, GRUB, gain, porn, plz]
1706    [Uber, discussing, buying, grub, thats, why, i...
1707    [Uber, puts, ., Grubhub, acquisition, gon, na,...
1708                                      [[, deleted, ]]
Name: tokenized_sents, Length: 1709, dtype: object


In [19]:
comments_stuff = data.drop(columns=['Unnamed: 0',"Author", 'tokenized_sents', 'Score', 
                                   'timestamp'])
comments_stuff.head()

,Comment
0,"all right, bears. its been fun, but im out"
1,Bloomberg - Markets Brace for Catastrophic Job...
2,Literally the worst jobs report in history goi...
3,magine returning to work to find out:\n\n1) sa...
4,I WAS PROMISED A TOTAL FINANCIAL COLLAPSE WHER...


In [32]:
vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(listt)

AttributeError: 'list' object has no attribute 'lower'

In [26]:
# Creating a DataFrame Representation of the TF-IDF results
money_news_df = pd.DataFrame(
    list(zip(vectorizer.get_feature_names(), np.ravel(X.sum(axis=0)))),
    columns=["Word", "Frequency"],
)

# Order the DataFrame by word frequency in descending order
money_news_df = money_news_df.sort_values(by=["Frequency"], ascending=False)

# Print the top 10 words
money_news_df.head(10)


,Word,Frequency
0,comment,1.0


In [25]:
comments_stuff.Comment.str.split(expand=True).stack().value_counts()

the          749
to           595
a            488
and          472
I            459
            ... 
79             1
devaluing      1
Financial      1
$108.47        1
retarded"      1
Length: 5800, dtype: int64

In [24]:
X

<1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [31]:
listt = comments_stuff.values.tolist()
listt

[['all right, bears. its been fun, but im out'],
 ['Bloomberg - Markets Brace for Catastrophic Jobs Report\n\ntranslation - Preparing the greatest pump of all time'],
 ['Literally the worst jobs report in history going to be reported this morning and were knocking on late 2019 levels. Cant make this shit up.'],
 ['magine returning to work to find out:\n\n1) salary is frozen indefinitely\n\n2) bonus is vaporized\n\n3) workload tremendous\n\n4) you made moar staying home the week before '],
 ['I WAS PROMISED A TOTAL FINANCIAL COLLAPSE WHERE ARE MY TENDIEEEEEEEEEEEEEEEEEEEEEEEEEEEEEESSSSS? '],
 ["From the jobs report\n\n**Average hourly earnings month on month:** \\+4.7% vs. +0.4% expected and +0.5% in March\n\n**Average hourly earnings year on year:** \\+7.9% vs. +3.3% expected and +3.3% in March\n\nis this just saying all the poor people were fired (average goes up because the low end of hourly earnings got removed) ?\n\nDoubt it's because people are getting massive payrises..."],
 ['Da

In [ ]:
# Create the Facebook Libra sentiment scores DataFrame
libra_sentiments = []

for article in libra_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        libra_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
libra_df = pd.DataFrame(libra_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
libra_df = libra_df[cols]

libra_df.head()

